In [698]:
# !pip install pyecharts
# !pip install selenium
import numpy as np 
import matplotlib.pyplot as plt 
#!pip install yfinance
import yfinance as yf # https://pypi.org/project/yfinance/
import math
import random
import seaborn as sns
import datetime
import pandas as pd
from pyecharts.charts import Bar, Line
from pyecharts import options as opts

In [865]:
# get data by ticker-name, start-time & end-time
def get_df_data(ticker_name="AAPL", start_time="2022-01-01", end_time="2022-10-09"):
  df_data = yf.download(tickers=ticker_name, start=start_time, end=end_time) 
  return df_data

def draw_2lines(x, y1, tn1, y2, tn2, ls1=50, le1=160, ls2=50, le2=160):
    data_1 = y1 #np.sin(x)
    data_2 = y2 #np.cos(x)
    # Create Plot
    fig, ax1 = plt.subplots(figsize=(25, 10)) 
    ax1.set_xlabel('Time range') 
    ax1.set_ylabel(tn1, color = 'red') 
    ax1.plot(x, data_1, color = 'red', label=tn1) 
    ax1.tick_params(axis ='y', labelcolor = 'red') 
    ax1.set_ylim(ls1, le1)
    ax2 = ax1.twinx() 
    ax2.set_ylabel(tn2, color = 'blue') 
    ax2.plot(x, data_2, color = 'blue', label=tn2) 
    ax2.tick_params(axis ='y', labelcolor = 'blue') 
    ax2.set_ylim(ls2, le2)
    # show the graph
    plt.legend()
    plt.show()
    return 

def time_point_dic(time_list, point_close):
    dic = {}
    i = 0
    while i<len(time_list):
        dic[time_list[i]] = point_close[i]
        i += 1
    return dic

# 
def data_union(dic1, dic2):
    # add in complement time values
    time1 = list(dic1.keys())
    time2 = list(dic2.keys())
    time_union = sorted(list(set(time1+time2)))
    for t in time_union:
        if t not in dic1:
            dic1[t] = -1
        if t not in dic2:
            dic2[t] = -1
    return time_union

def bin_analysis(point_list, low_level=0.0, high_level=3.0, level_width=0.1):
    # low_level, high_level, level_width, 
    k_num = round((high_level-low_level)/level_width) + 1
    N = len(point_list)
    fre_dic = {}
    for k in range(k_num):
        fre_dic[round(k*level_width, 1)] = 0

    for v in point_list:
        k = round(v/level_width) * level_width
        if k>high_level:
            k = high_level
        k = round(k, 1)
        fre_dic[k] += 1

    cumulative = 0
    for k in range(k_num):
        k = round(k*level_width, 1)
        cumulative = cumulative + fre_dic[k]
        r = round(cumulative/N*100, 2)
        print( k, "\t", cumulative, "\t", r, "%" )
    return 

def find_previous_businessday(i, x, y):
    counter = i
    t = x[counter]
    p = y[counter]
    while p<0 and counter>0:
        t = x[counter]
        p = y[counter]
        counter = counter - 1
    #if counter!=i:
        #print( "\t reference stock in holiday \t", x[i], y[i], " -> ", t, p )
    return t, p

def find_next_businessday(i, j, x, y):
    counter = i
    point_list = []
    while counter<=j and counter<len(x):
        point_list.append( y[counter] )
        counter += 1
    if len(point_list)==0:
        return 0
    return max(point_list)

# 
def back_test(signal, in_days, x, 
              y1_open, y1_close, y1_low, y1_high, 
              y2_open, y2_close, y2_low, y2_high):
    profit_list = []
    for i in range(0, len(x)-1):
            # reference stock info
        reference_t, reference_p = find_previous_businessday(i, x, y1_close)
            # target stock info
        target_t = x[i+1]
        target_open = y2_open[i+1]
        target_close = y2_close[i+1]
        target_low = y2_low[i+1]
        target_high = y2_high[i+1]
        if target_open<0:
            #print( target_t, "\t target stock in holiday" )
            continue
        #print( target_t, target_open, target_close, target_low, target_high, "\t", reference_t, reference_p )
            # possible profit
        if target_low<reference_p-signal:
            buy_point = min(target_open, reference_p-2)
            j = i+in_days
            sell_point = max( target_close, find_next_businessday(i+2, j, x, y2_high) )
            profit = round( sell_point-buy_point, 2 )
            profit_list.append( profit )
    return profit_list

def get_list(from_num=1.5, to_num=3.0, step_num=0.1):
    tmp_list = []
    num = from_num
    while num<=to_num:
        num = round(num, 1)
        tmp_list.append( num )
        num += step_num
    return tmp_list

def profit_analysis(profit_list, from_num=1.0, to_num=3.0, step_num=0.1):
    info = []
    N = len(profit_list)
    for profit_per_share in get_list(from_num, to_num, step_num):
        counter = 0
        for v in profit_list:
            if v >= profit_per_share:
                counter += 1
        r = round(counter / N * 100, 2)
        info.append( (profit_per_share, N, counter, r) )
    return info

#
# data_info, 
    # signal
        # in_days
            # profit_per_share, ratio
def data_by_signal(data_info, ll=2.0, hl=11):
    data, col_names = [], []
    for signal in range(hl):
        signal = ll + 0.1*signal
        signal = round(signal, 1)
        #
        info = data_info[signal]        
        for days in sorted( list(info.keys()) ):
            it = info[days]
            N = len(it)
            tmp_n = ["signal", "days"]
            tmp_v = [signal, days]   
            i = 0
            while i<N:
                tmp_n.append( "profit_" + str(it[i][0]) )
                tmp_v.append( it[i][1] )            
                i = i + 1
            data.append(tmp_v)
            col_names.append( tmp_n )
    col_names = col_names[0]
    #print(col_names)
    df = pd.DataFrame(data, columns=col_names)
    return df



In [747]:
st, et = "2022-02-01", "2023-01-31"
print("time range:\t", st, "-", et)

tn1 = "BABA" # BABA, BIDU
df_data1 = get_df_data(ticker_name=tn1, start_time=st, end_time=et)
df_data1 = df_data1[ ["Open", "High", "Low", "Close", "Volume"] ]
print("stock:\t", tn1, "\t")
print(df_data1)

tn2 = "9988.HK" # 9988.HK, 9888.HK
df_data2 = get_df_data(ticker_name=tn2, start_time=st, end_time=et)
df_data2 = df_data2[ ["Open", "High", "Low", "Close", "Volume"] ]
print("stock:\t", tn2, "\t")
print(df_data2)

# dict-type: key -> datetime, value -> point
dic1_open = time_point_dic(list(df_data1.index), list(df_data1['Open']))
dic1_close = time_point_dic(list(df_data1.index), list(df_data1['Close']))
dic1_low = time_point_dic(list(df_data1.index), list(df_data1['Low']))
dic1_high = time_point_dic(list(df_data1.index), list(df_data1['High']))

dic2_open = time_point_dic(list(df_data2.index), list(df_data2['Open']))
dic2_close = time_point_dic(list(df_data2.index), list(df_data2['Close']))
dic2_low = time_point_dic(list(df_data2.index), list(df_data2['Low']))
dic2_high = time_point_dic(list(df_data2.index), list(df_data2['High']))
print( "2 stocks:\t")
print( "\t", len(dic1_open), len(dic1_close), len(dic1_low), len(dic1_high) )
print( "\t", len(dic2_open), len(dic2_close), len(dic2_low), len(dic2_high) )

# data union, stock 1 may have some business-day when stock 2 in holiday, and vice versa
x1 = data_union(dic1_open, dic2_open)
x2 = data_union(dic1_close, dic2_close)
x3 = data_union(dic1_low, dic2_low)
x4 = data_union(dic1_high, dic2_high)
assert len(x1)==len(x2)==len(x3)==len(x4)
print( "2 stocks:\t")
print( "\t", len(dic1_open), len(dic1_close), len(dic1_low), len(dic1_high) )
print( "\t", len(dic2_open), len(dic2_close), len(dic2_low), len(dic2_high) )

# split dict-type data into x & y list-type following the datetime order
x = x1
y1_open, y1_close, y1_low, y1_high = [], [], [], []
y2_open, y2_close, y2_low, y2_high = [], [], [], []
for t in x:
    y1_open.append( round(dic1_open[t], 2) )
    y1_close.append( round(dic1_close[t], 2) )
    y1_low.append( round(dic1_low[t], 2) )
    y1_high.append( round(dic1_high[t], 2) )
    #
    y2_open.append( round(dic2_open[t], 2) )
    y2_close.append( round(dic2_close[t], 2) )
    y2_low.append( round(dic2_low[t], 2) )
    y2_high.append( round(dic2_high[t], 2) )
print( "2 stocks:\t", len(x) )
print( "\t", len(y1_open), len(y1_close), len(y1_low), len(y1_high) )
print( "\t", len(y2_open), len(y2_close), len(y2_low), len(y2_high) )

time range:	 2022-02-01 - 2023-01-31
[*********************100%***********************]  1 of 1 completed
stock:	 BABA 	
                  Open        High         Low       Close    Volume
Date                                                                
2022-02-01  124.809998  128.000000  122.970001  127.220001  15036100
2022-02-02  127.629997  127.629997  121.169998  122.879997  14050100
2022-02-03  121.550003  125.269997  120.730003  123.470001  16651500
2022-02-04  121.269997  123.940002  119.660004  122.220001  13538200
2022-02-07  116.099998  116.550003  113.599998  114.820000  28589200
...                ...         ...         ...         ...       ...
2022-12-23   87.849998   88.650002   85.519997   85.650002  17949300
2022-12-27   86.550003   90.689003   86.209999   89.860001  22959900
2022-12-28   88.980003   90.150002   86.464996   87.199997  16268800
2022-12-29   87.625000   89.519997   87.059998   89.129997  12535400
2022-12-30   87.449997   89.410004   87.410004   88

In [871]:
#
data_info = {}
signal_list = get_list(1.5, 3.0, 0.1)
for signal in signal_list:
    data_info[signal] = {}
    for in_days in range(2, 11):
        data_info[signal][in_days] = []
        profit_list = back_test(signal, in_days, x, y1_open, y1_close, y1_low, y1_high, y2_open, y2_close, y2_low, y2_high)
        profit_info = profit_analysis(profit_list, 1.0, 3.0, 0.2)
        for it in profit_info:
            profit_per_share, N, _, ratio = it
            data_info[signal][in_days].append( (profit_per_share, ratio, N) )

info_df = data_by_signal(data_info, 1.5, 16)
tmp_df = info_df.sort_values(by=['signal', 'days'])
tmp_df.to_csv("C:/Users/Admin/Desktop/stocks_analyze_predict/stock_analysis01.csv")
tmp_df = info_df.sort_values(by=['days', 'signal'])
tmp_df.to_csv("C:/Users/Admin/Desktop/stocks_analyze_predict/stock_analysis02.csv")
info_df

,signal,days,profit_1.0,profit_1.2,profit_1.4,profit_1.6,profit_1.8,profit_2.0,profit_2.2,profit_2.4,profit_2.6,profit_2.8,profit_3.0
0,1.5,2,58.74,55.83,52.91,49.51,45.63,43.69,41.75,39.32,36.41,35.44,34.47
1,1.5,3,69.42,66.99,64.08,62.14,58.25,56.31,53.88,50.49,48.06,47.09,45.63
2,1.5,4,74.27,71.84,68.45,66.02,63.59,62.14,60.68,57.28,54.85,52.91,51.94
3,1.5,5,76.21,73.79,70.87,68.93,66.99,65.53,64.08,61.17,59.22,58.25,56.80
4,1.5,6,77.67,75.24,72.82,70.87,68.45,66.99,65.53,63.59,62.14,61.17,60.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,3.0,6,69.92,68.29,67.48,66.67,63.41,61.79,59.35,56.91,56.91,55.28,54.47
140,3.0,7,72.36,69.92,69.11,68.29,65.85,65.04,62.60,58.54,58.54,56.91,55.28
141,3.0,8,74.80,72.36,71.54,70.73,69.11,68.29,65.85,61.79,60.98,59.35,57.72
142,3.0,9,76.42,73.98,73.17,72.36,70.73,69.92,67.48,64.23,64.23,62.60,60.98


In [868]:
info_df[info_df['days']==5]

,signal,days,profit_1.0,profit_1.2,profit_1.4,profit_1.6,profit_1.8,profit_2.0,profit_2.2,profit_2.4,profit_2.6,profit_2.8,profit_3.0
3,1.5,5,76.21,73.79,70.87,68.93,66.99,65.53,64.08,61.17,59.22,58.25,56.80
12,1.6,5,75.38,72.86,69.85,68.34,66.83,65.33,63.82,60.80,58.79,57.79,56.28
21,1.7,5,75.13,72.59,69.54,68.02,66.50,64.97,63.45,60.41,58.38,57.36,55.84
30,1.8,5,75.26,72.68,69.59,68.04,66.49,64.95,63.40,60.31,58.25,57.22,55.67
39,1.9,5,74.74,72.11,69.47,67.89,66.32,64.74,63.16,60.00,57.89,56.84,55.26
48,2.0,5,74.33,71.66,68.98,67.38,65.78,64.17,62.57,59.36,57.22,56.15,54.55
57,2.1,5,73.18,70.39,68.16,66.48,64.80,63.13,61.45,58.66,56.98,55.87,54.19
66,2.2,5,72.88,70.06,67.80,66.10,64.41,62.71,61.02,58.19,56.50,55.37,53.67
75,2.3,5,72.09,69.19,66.86,65.12,63.37,61.63,59.88,56.98,55.81,54.65,52.91
84,2.4,5,70.91,67.88,65.45,64.24,62.42,61.21,59.39,56.36,55.15,53.94,52.12


In [874]:
info_df[info_df['signal']==1.5]

,signal,days,profit_1.0,profit_1.2,profit_1.4,profit_1.6,profit_1.8,profit_2.0,profit_2.2,profit_2.4,profit_2.6,profit_2.8,profit_3.0
0,1.5,2,58.74,55.83,52.91,49.51,45.63,43.69,41.75,39.32,36.41,35.44,34.47
1,1.5,3,69.42,66.99,64.08,62.14,58.25,56.31,53.88,50.49,48.06,47.09,45.63
2,1.5,4,74.27,71.84,68.45,66.02,63.59,62.14,60.68,57.28,54.85,52.91,51.94
3,1.5,5,76.21,73.79,70.87,68.93,66.99,65.53,64.08,61.17,59.22,58.25,56.80
4,1.5,6,77.67,75.24,72.82,70.87,68.45,66.99,65.53,63.59,62.14,61.17,60.19
5,1.5,7,79.61,77.18,74.76,72.82,70.87,69.90,68.45,66.02,64.56,63.59,62.14
6,1.5,8,81.07,78.64,76.21,74.27,72.82,71.84,70.39,67.96,66.50,65.53,64.08
7,1.5,9,82.04,79.61,77.18,75.73,74.27,73.30,71.84,69.90,68.93,67.96,66.50
8,1.5,10,83.01,80.58,78.64,77.18,76.21,75.24,73.79,71.84,70.87,69.90,68.45


In [883]:
line = (
    Line(init_opts=opts.InitOpts(width="1800px", height="900px"))    
    .set_global_opts(title_opts=opts.TitleOpts(title="stocks analysis"), 
                     tooltip_opts=opts.TooltipOpts(trigger="axis"),
                    yaxis_opts=opts.AxisOpts(name='percentage',splitline_opts=opts.SplitLineOpts(is_show=True),min_=0))# , subtitle="商店A中六樣商品數"
)

x_values = []
for v in range(2, 11):
    x_values.append( str(v) + " days" )
line.add_xaxis( x_values )

for signal in signal_list:
    signal = round(signal, 1)
    html_df = info_df[ info_df['signal']==signal ]
    #x_values = html_df['days']
    y_values = html_df['profit_2.0']
    line.add_yaxis( str(signal) + " signal", y_values )

line.render("lines_stock_analysis01.html")

'C:\\Users\\Admin\\lines_stock_analysis01.html'

In [690]:











    
    



"""
# data visualization
ls1=ls2=60 
le1=le2=100

y1, y2 = y1_open, y2_open
draw_2lines(x, y1, tn1, y2, tn2, ls1, le1, ls2, le2)


# data visualization: file:///C:/Users/Admin/lines.html
line = (
    Line(init_opts=opts.InitOpts(width="2500px", height="1200px"))
    .add_xaxis(x)
    .add_yaxis(tn1+"_open", y1_open)
    .add_yaxis(tn1+"_close", y1_close)
    .add_yaxis(tn1+"_low", y1_low)
    .add_yaxis(tn1+"_high", y1_high)
    .add_yaxis(tn2+"_open", y2_open)
    .add_yaxis(tn2+"_close", y2_close)
    .add_yaxis(tn2+"_low", y2_low)
    .add_yaxis(tn2+"_high", y2_high)
    .set_global_opts(title_opts=opts.TitleOpts(title="stocks comparison"), 
                     tooltip_opts=opts.TooltipOpts(trigger="axis"),
                    yaxis_opts=opts.AxisOpts(name='dollar',splitline_opts=opts.SplitLineOpts(is_show=True),min_=60))# , subtitle="商店A中六樣商品數"
    
)
line.render("lines.html")

x_update = x[:-1]
y1_close_update = y1_close[:-1]
y2_open_update = y2_open[1:]
y2_close_update = y2_close[1:]
y2_low_update = y2_low[1:]
y2_high_update = y2_high[1:]

line_update = (
    Line(init_opts=opts.InitOpts(width="1800px", height="960px"))
    .add_xaxis(x_update)
    #.add_yaxis(tn1+"_open", y1_open)
    .add_yaxis(tn1+"_close", y1_close_update)
    #.add_yaxis(tn1+"_low", y1_low)
    #.add_yaxis(tn1+"_high", y1_high)
    .add_yaxis(tn2+"_open", y2_open_update)
    .add_yaxis(tn2+"_close", y2_close_update)
    .add_yaxis(tn2+"_low", y2_low_update)
    .add_yaxis(tn2+"_high", y2_high_update)
    .set_global_opts(title_opts=opts.TitleOpts(title="stocks comparison"), 
                     tooltip_opts=opts.TooltipOpts(trigger="axis"),
                    yaxis_opts=opts.AxisOpts(name='dollar',splitline_opts=opts.SplitLineOpts(is_show=True),min_=60))# , subtitle="商店A中六樣商品數"
    
)
line_update.render("lines_update.html")
"""

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
2 stocks:	
	 84 84 84 84
	 83 83 83 83
	 86 86 86 86
	 86 86 86 86
3.3 2022-09-05 00:00:00
3.2 2022-09-07 00:00:00
2.32 2022-09-08 00:00:00
2.34 2022-09-09 00:00:00
4.89 2022-09-12 00:00:00
3.96 2022-09-19 00:00:00
6.29 2022-09-28 00:00:00
2.21 2022-09-29 00:00:00
2.04 2022-09-30 00:00:00
2.8 2022-10-03 00:00:00
6.46 2022-10-04 00:00:00
2.26 2022-10-13 00:00:00
3.57 2022-10-17 00:00:00
2.12 2022-10-20 00:00:00
2.58 2022-10-21 00:00:00
6.21 2022-10-26 00:00:00
2.13 2022-10-31 00:00:00
2.21 2022-11-03 00:00:00
6.77 2022-11-10 00:00:00
6.01 2022-11-17 00:00:00
2.89 2022-11-22 00:00:00
3.1 2022-11-23 00:00:00
2.1 2022-11-24 00:00:00
3.68 2022-11-28 00:00:00
7.66 2022-11-30 00:00:00
3.14 2022-12-01 00:00:00
5.26 2022-12-02 00:00:00
3.28 2022-12-07 00:00:00
4.77 2022-12-08 00:00:00
3.06 2022-12-13 00:00:00
3.36 2022-12-21 00:00:00
3.36 2022-12-27 00:00:00


'C:\\Users\\Admin\\lines_update.html'

In [875]:
# 
import datetime
def get_dates_from_range(start_date, end_dates):
    start = datetime.datetime.strptime(start_date, "%d-%m-%Y") 
    end = datetime.datetime.strptime(end_dates, "%d-%m-%Y")
    date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]
    date_list = []
    for date in date_generated:
        date_ymd = date
        y, m, d = date_ymd.year, date_ymd.month, date_ymd.day
        ymd = str(y) + "-"
        if m<10:
            ymd = ymd + "0" + str(m)
        else:
            ymd = ymd + str(m)
        ymd = ymd + "-"
        if d<10:
            ymd = ymd + "0" + str(d)
        else:
            ymd = ymd + str(d)
        date_list.append( ymd  )
    return date_list

def open_close_data(tn, st, et):
    tmp_df = get_df_data(ticker_name=tn, start_time=st, end_time=et)
    if len(tmp_df)>0:
        it = tmp_df.iloc[0]
        return it['Open'], it['Close']
    return 0, 0

#print(start_time, end_time)

In [876]:
ticker_name = '9988.HK'
start_time = '2022-12-19'
end_time = '2022-12-20'

date_list = get_dates_from_range('30-12-2022', '05-01-2023')
date_list

df_list = []

nn = 0
while nn<len(date_list)-1: # 
    start_time, end_time = date_list[nn], date_list[nn+1]
    #
    interval_len = '1m'
    df_data = yf.download(tickers=ticker_name, start=start_time, end=end_time, interval=interval_len)
    if len(df_data)>0:
        max( list(df_data['Close']) ), max( list(df_data['Open']) ), max( list(df_data['Low']) ), max( list(df_data['High']) )

        info_list = []
        i = 0
        open_point, close_point = open_close_data(ticker_name, start_time, end_time)
        while i<len(df_data):
            current_time = df_data.index[i]
            current_it = df_data.iloc[i]
            current_point = current_it['Close']
                # initialization
            future_max_time = current_time
            future_max_point = current_point 
            j = i + 1
            while j<len(df_data):
                future_time = df_data.index[j]
                future_it = df_data.iloc[j]
                future_point = future_it['Close']
                if future_max_point<future_point:
                    future_max_point = future_point
                    future_max_time = future_time
                j += 1
            ratio = (future_max_point - current_point)/current_point * 100
            #print( current_time, round(current_point, 2), future_max_time, round(future_max_point, 2), '\t', round(ratio, 2) )
            info_list.append( [open_point, close_point, current_time, current_point, future_max_time, future_max_point, ratio] )
            i += 1
        new_df = pd.DataFrame(info_list, columns = ['open_point', 'close', 'current_time', 'current_point', 'future_max_time', 'future_max_point', 'ratio'])

        max_info = new_df.sort_values(by=['ratio'], ascending=False)
        print(start_time, end_time, "\t")
        #print(max_info.head(3))
        #print()
        df_list.append( max_info.head(3) )
    nn += 1

useful_info = pd.concat( df_list )
useful_info
#useful_info.to_csv('C:/Users/Admin/Desktop/stocks_analyze_predict/daily_profit.csv')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
2022-12-30 2022-12-31 	
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 9988.HK: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 9988.HK: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 9988.HK: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
2023-01-03 2023-01-04 	


,open_point,close,current_time,current_point,future_max_time,future_max_point,ratio
71,87.300003,86.250000,2022-12-30 10:41:00,85.849998,2022-12-30 13:12:00,87.000000,1.339548
73,87.300003,86.250000,2022-12-30 10:43:00,85.900002,2022-12-30 13:12:00,87.000000,1.280557
72,87.300003,86.250000,2022-12-30 10:42:00,85.949997,2022-12-30 13:12:00,87.000000,1.221644
21,85.699997,88.650002,2023-01-03 09:51:00,84.650002,2023-01-03 14:15:00,89.099998,5.256937
22,85.699997,88.650002,2023-01-03 09:52:00,84.699997,2023-01-03 14:15:00,89.099998,5.194807
20,85.699997,88.650002,2023-01-03 09:50:00,84.750000,2023-01-03 14:15:00,89.099998,5.132742


In [881]:
ratio_list = []

ratio_list.append( 2.05/82.95*100 )
ratio_list.append( 2.35/85.75*100 )
ratio_list.append( 2.5/85*100 )

sum(ratio_list) - 0.5*len(ratio_list)

6.653069546082447